# Calliope Urban Scale MILP Example Model 

For more details on analysing input/output data, see the full urban scale example model

In [1]:
import calliope

# cufflinks allows for easy plotly plots to be 
# produced from a pandas DataFrame
import cufflinks

# We increase logging verbosity
calliope.set_log_level('INFO')

In [2]:
model = calliope.examples.milp()

# Note, we see the overrides that we have applied printed here, thanks to inreasing logging verbosity

[2018-04-20 09:51:37] INFO: Model: initialising
[2018-04-20 09:51:38] INFO: Override applied to model.name: Urban-scale example model -> Urban-scale example model with MILP
`techs.boiler.costs.monetary.energy_cap`:35 applied from override as new configuration
`techs.boiler.costs.monetary.purchase`:2000 applied from override as new configuration
`techs.chp.constraints.energy_cap_min_use`:0.2 applied from override as new configuration
`techs.chp.constraints.energy_cap_per_unit`:300 applied from override as new configuration
`techs.chp.constraints.units_max`:4 applied from override as new configuration
Override applied to techs.chp.costs.monetary.energy_cap: 750 -> 700
`techs.chp.costs.monetary.purchase`:40000 applied from override as new configuration
[2018-04-20 09:51:39] INFO: Model: preprocessing stage 1 (model_run)
[2018-04-20 09:51:41] INFO: Model: preprocessing stage 2 (model_data)
[2018-04-20 09:51:43] INFO: Model: preprocessing complete. Time since start: 0:00:05.708900


In [3]:
# Model inputs can be viewed at `model.inputs`. 
# Variables are indexed over any combination of `techs`, `locs`, `carriers`, `costs` and `timesteps`, 
# although `techs`, `locs`, and `carriers` are often concatenated. 
# e.g. `chp`, `X1`, `heat` -> `X1::chp::heat` 
model.inputs

<xarray.Dataset>
Dimensions:                            (carrier_tiers: 3, carriers: 3, coordinates: 2, costs: 1, loc_carriers: 10, loc_tech_carriers_conversion_plus: 3, loc_techs: 26, loc_techs_area: 3, loc_techs_conversion: 2, loc_techs_conversion_plus: 1, loc_techs_export: 4, loc_techs_finite_resource: 9, loc_techs_investment_cost: 20, loc_techs_milp: 1, loc_techs_non_conversion: 23, loc_techs_om_cost: 7, loc_techs_supply_plus: 3, loc_techs_transmission: 10, locs: 4, techs: 9, timesteps: 48)
Coordinates:
  * loc_techs_finite_resource          (loc_techs_finite_resource) object 'X3::demand_heat' ...
  * costs                              (costs) object 'monetary'
  * loc_techs_area                     (loc_techs_area) object 'X3::pv' ...
  * loc_techs_non_conversion           (loc_techs_non_conversion) object 'X3::demand_heat' ...
  * timesteps                          (timesteps) datetime64[ns] 2005-07-01 ...
  * loc_techs                          (loc_techs) object 'X3::demand_heat

In [4]:
# Individual data variables can be accessed easily, `to_pandas()` reformats the data to look nicer
# Here we look at one of the MILP overrides that we have added, the fixed `purchase` cost
model.inputs.cost_purchase.to_pandas().dropna(axis=1)

loc_techs_investment_cost,X3::boiler,X2::boiler,X1::chp
costs,,,
monetary,2000.0,2000.0,40000.0


In [5]:
# Solve the model. Results are loaded into `model.results`. 
# By including logging (see package importing), we can see the timing of parts of the run, as well as the solver's log
model.run()

[2018-04-20 09:51:45] INFO: Backend: starting model run
[2018-04-20 09:51:47] INFO: Backend: model generated. Time since start: 0:00:09.997301
[2018-04-20 09:51:47] INFO: Backend: sending model to solver
[2018-04-20 09:51:52] INFO: Backend: solver finished running. Time since start: 0:00:14.669407
[2018-04-20 09:51:52] INFO: Backend: loaded results
[2018-04-20 09:51:52] INFO: Backend: generated solution array. Time since start: 0:00:14.950685
[2018-04-20 09:51:52] INFO: Postprocessing: started
[2018-04-20 09:51:53] INFO: Postprocessing: All values < 1e-10 set to 0 in unmet_demand
[2018-04-20 09:51:53] INFO: Postprocessing: Model was feasible, deleting unmet_demand variable
[2018-04-20 09:51:53] INFO: Postprocessing: ended. Time since start: 0:00:15.778898


In [6]:
# Model results are held in the same structure as model inputs. 
# The results consist of the optimal values for all decision variables, including capacities and carrier flow
# There are also results, like system capacity factor and levelised costs, which are calculated in postprocessing
# before being added to the results Dataset

model.results

<xarray.Dataset>
Dimensions:                     (carriers: 3, costs: 1, loc_tech_carriers_con: 19, loc_tech_carriers_export: 4, loc_tech_carriers_prod: 21, loc_techs: 26, loc_techs_area: 3, loc_techs_cost: 20, loc_techs_investment_cost: 20, loc_techs_milp: 1, loc_techs_om_cost: 7, loc_techs_purchase: 2, loc_techs_supply_plus: 3, techs: 9, timesteps: 48)
Coordinates:
  * loc_tech_carriers_export    (loc_tech_carriers_export) object 'X1::pv::electricity' ...
  * costs                       (costs) object 'monetary'
  * loc_techs_cost              (loc_techs_cost) object 'X2::heat_pipes:N1' ...
  * loc_techs_area              (loc_techs_area) object 'X3::pv' 'X2::pv' ...
  * timesteps                   (timesteps) datetime64[ns] 2005-07-01 ...
  * loc_techs                   (loc_techs) object 'X3::demand_heat' ...
  * loc_techs_milp              (loc_techs_milp) object 'X1::chp'
  * techs                       (techs) object 'supply_grid_power' 'boiler' ...
  * loc_tech_carriers_prod   

In [7]:
# We can sum operating units of CHP over all locations and turn the result into a pandas DataFrame
df_units = model.get_formatted_array('operating_units').sum('locs').to_pandas().T

#The information about the dataframe tells us about the amount of data it holds in the index and in each column
df_units.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 48 entries, 2005-07-01 00:00:00 to 2005-07-02 23:00:00
Data columns (total 1 columns):
chp    48 non-null float64
dtypes: float64(1)
memory usage: 768.0 bytes


In [8]:
# Using .head() to see the first few rows of operating units

df_units.head()

techs,chp
timesteps,
2005-07-01 00:00:00,1.0
2005-07-01 01:00:00,1.0
2005-07-01 02:00:00,1.0
2005-07-01 03:00:00,1.0
2005-07-01 04:00:00,1.0


In [9]:
# We can plot this by using the timeseries plotting functionality.
# The top-left dropdown gives us the chance to scroll through other timeseries data too.

model.plot.timeseries()

In [10]:
# plot.capacities gives a graphical view of the non-timeseries variables, both input and output

# Note, because we fix unit size, CHP now has a maximum capacity of 300kW, 
# compared to 260kW in the non-MILP case

model.plot.capacity()

See the [Calliope documentation](https://calliope.readthedocs.io/) for more details on setting up and running a Calliope model.